In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

In [3]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [5]:
# Setup for train dataset

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)

In [6]:
model = keras.Sequential(
    [
        keras.Input((28,28,1)),
        layers.Conv2D(32,3,activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax")
    ]
)

num_epochs = 5
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [8]:
# Training Loop
for epoch in range(num_epochs):
    print(f"\nStart of Training Epoch {epoch}")
    for batch_idx, (x_batch, y_batch) in enumerate(ds_train):
        with tf.GradientTape() as tape:
            y_pred = model(x_batch, training=True)
            loss = loss_fn(y_batch, y_pred)
        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        acc_metric.update_state(y_batch, y_pred)
    train_acc = acc_metric.result()
    print(f"Accuracy over epoch {train_acc}")
    acc_metric.reset_states()



Start of Training Epoch 0
loss: 
tf.Tensor(0.034672797, shape=(), dtype=float32)
gradients: 
[<tf.Tensor: shape=(3, 3, 1, 32), dtype=float32, numpy=
array([[[[-8.65637790e-04,  2.26055202e-03,  3.50832846e-03,
           4.91480436e-03, -1.56615383e-03, -8.75740685e-03,
           3.40196327e-03, -8.64233728e-03, -2.48442143e-02,
           5.55068068e-03,  8.90914933e-04, -8.29475187e-03,
          -1.05256215e-02,  2.47244863e-03,  7.82457739e-03,
           8.90896842e-03, -4.42322670e-03,  2.44406471e-03,
           2.09391490e-03, -2.64645293e-02,  9.86581901e-04,
           1.37821119e-03,  5.77009516e-03,  1.23561604e-03,
          -3.35957631e-02,  2.24249088e-04,  3.74083268e-03,
           1.67433657e-02,  2.38085678e-03,  1.94094772e-03,
          -1.64190754e-02, -1.86736137e-02]],

        [[-4.89885546e-03, -5.06131910e-05,  2.03731610e-03,
           2.94287805e-03, -8.04961310e-04, -1.42226517e-02,
           3.59267695e-03, -1.15577038e-02, -1.65257957e-02,
          

KeyboardInterrupt: 

In [ ]:
# Test Loop
for batch_idx, (x_batch, y_batch) in enumerate(ds_test):
    y_pred = model(x_batch, training=False)
    acc_metric.update_state(y_batch, y_pred)

train_acc = acc_metric.result()
print(f"Accuracy over Test Set: {train_acc}")
acc_metric.reset_states()

Accuracy over TEst Set: 0.9921500086784363
